# AutoShorts 모듈별 테스트 노트북

각 Agent를 개별적으로 테스트하고 프롬프트를 확인할 수 있습니다.

## 📋 필요한 API 키

`.env` 파일에 다음 키들을 설정하세요:

```bash
# 필수
OPENAI_API_KEY=sk-...           # GPT-4o-mini (플롯 생성)
REDIS_URL=redis://localhost:6379

# 권장
ELEVENLABS_API_KEY=...          # ElevenLabs (음성+음악 생성, $5/월~)
GEMINI_API_KEY=...              # Gemini 2.5 Flash Image (나노바나나, 이미지 생성)

# 선택 (없으면 Stub 모드로 동작)
COMFYUI_URL=http://localhost:8188  # ComfyUI (이미지 생성, 로컬 무료)
MUBERT_LICENSE=...              # Mubert (음악 폴백, $14/월~, deprecated)
PLAYHT_USER_ID=...              # PlayHT (음성 대체, $31/월~)
PLAYHT_API_KEY=...
```

**비용 최적화 팁:**
- ElevenLabs 하나로 음성+음악 모두 생성 가능 ($5/월 플랜)
- Gemini API로 이미지 생성 (나노바나나, $0.039/이미지)
- Mubert는 비쌈 ($14/월), ElevenLabs로 대체 권장
- ComfyUI는 로컬 실행 시 무료 (GPU 필요)

## 🚀 실행 전 체크리스트

1. Redis 서버 실행: `redis-server`
2. ComfyUI 실행 (선택): `cd ComfyUI && python main.py`
3. 가상환경 활성화: `source kvenv/bin/activate`
4. 필요한 패키지 설치 확인: `pip install -r backend/requirements.txt`

## 0. 환경 설정

In [1]:
!pip install -r backend/requirements.txt


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# Python path 설정 (backend 모듈 import 가능하도록)
import sys
from pathlib import Path

backend_path = Path.cwd() / "backend"
if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))

print(f"✅ Python path 설정 완료: {backend_path}")

✅ Python path 설정 완료: /Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI/backend


In [3]:
# 필요한 모듈 import
import json
from datetime import datetime
from pathlib import Path

print("✅ 기본 모듈 import 완료")

✅ 기본 모듈 import 완료


In [4]:
# API 키 확인
from app.config import settings

print("=" * 50)
print("API 키 상태 확인")
print("=" * 50)

api_status = {
    "OpenAI (플롯 생성)": "✅" if settings.OPENAI_API_KEY else "❌",
    "Gemini (이미지 - 나노바나나)": "✅" if settings.GEMINI_API_KEY else "⚠️  Stub 모드",
    "ComfyUI (이미지 대체)": "✅" if settings.COMFY_URL else "⚠️  Stub 모드",
    "ElevenLabs (음성+음악)": "✅" if settings.ELEVENLABS_API_KEY else "⚠️  Stub 모드",
    "Mubert (음악 폴백)": "✅" if settings.MUBERT_LICENSE else "⚠️  Stub 모드",
    "PlayHT (음성 대체)": "✅" if settings.PLAYHT_API_KEY else "⚠️  Stub 모드",
    "Redis": "✅" if settings.REDIS_URL else "❌",
}

for service, status in api_status.items():
    print(f"{service:30s}: {status}")

print("\n💡 권장 설정:")
print("  - GEMINI_API_KEY: 이미지 생성 (나노바나나, $0.039/이미지)")
print("  - ELEVENLABS_API_KEY: 음성+음악 통합 (저렴, $5/월~)")
print("  - MUBERT_LICENSE: 음악 전용 폴백 (비쌈, $14/월~)")
print("\n⚠️  Stub 모드: API 키가 없으면 더미 데이터 생성")

API 키 상태 확인
OpenAI (플롯 생성)                : ✅
Gemini (이미지 - 나노바나나)          : ✅
ComfyUI (이미지 대체)              : ✅
ElevenLabs (음성+음악)            : ✅
Mubert (음악 폴백)                : ⚠️  Stub 모드
PlayHT (음성 대체)                : ⚠️  Stub 모드
Redis                         : ✅

💡 권장 설정:
  - GEMINI_API_KEY: 이미지 생성 (나노바나나, $0.039/이미지)
  - ELEVENLABS_API_KEY: 음성+음악 통합 (저렴, $5/월~)
  - MUBERT_LICENSE: 음악 전용 폴백 (비쌈, $14/월~)

⚠️  Stub 모드: API 키가 없으면 더미 데이터 생성


## 1. 플롯 생성 테스트 (스키마 v3: characters.json + plot.json → layout.json)

GPT-4o-mini를 사용하여:
1. **캐릭터 정의 생성** (characters.json): 외형, 성격, seed
2. **플롯 생성** (plot.json): 씬별 대사, 표정, 포즈
3. **JSON 변환** (layout.json): appearance + expression + pose 조합

### 스키마 v3 주요 변경사항
- ✅ **캐릭터 정의 분리**: characters.json (재사용 가능)
- ✅ **expression, pose 추가**: 씬별 세밀한 이미지 제어
- ✅ **JSON 형식**: CSV 대신 plot.json 사용 (파싱 문제 해결)
- 🎨 **이미지 생성**: appearance + expression + pose 자동 조합

In [5]:
# 테스트 파라미터 설정
test_prompt = "우주를 여행하는 용감한 고양이의 모험"
num_characters = 2
num_cuts = 5
mode = "story"

# Run ID 생성 (실제와 동일한 형식)
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
prompt_clean = "".join(c for c in test_prompt if not c.isspace())[:8]
run_id = f"{timestamp}_{prompt_clean}"

print(f"Run ID: {run_id}")
print(f"프롬프트: {test_prompt}")
print(f"캐릭터 수: {num_characters}")
print(f"장면 수: {num_cuts}")
print(f"모드: {mode}")

Run ID: 20251105_1738_우주를여행하는용
프롬프트: 우주를 여행하는 용감한 고양이의 모험
캐릭터 수: 2
장면 수: 5
모드: story


In [6]:
# 캐릭터 및 플롯 생성 (스키마 v3)
from app.utils.plot_generator import generate_plot_with_characters

print("=" * 50)
print("GPT-4o-mini에게 전달되는 프롬프트 (2단계)")
print("=" * 50)

# 1단계: 캐릭터 정의 생성
char_system_prompt = f"""당신은 숏폼 영상 콘텐츠의 캐릭터 디자이너입니다.
사용자의 요청에 맞는 {num_characters}명의 캐릭터를 만들어주세요.

각 캐릭터마다 다음 정보를 JSON 형식으로 제공하세요:
- char_id: char_1, char_2, ... 형식
- name: 캐릭터 이름 (창의적으로)
- appearance: 외형 묘사 (이미지 생성에 사용됩니다. 상세하게 작성)
- personality: 성격/특징
- voice_profile: "default" (고정값)
- seed: char_1은 1002, char_2는 1003, ... 순서대로

**중요**:
- 반드시 JSON 형식으로만 출력하세요
- appearance는 이미지 생성 프롬프트로 사용되므로 시각적 특징을 상세히 작성
"""

print("[1단계: 캐릭터 정의 생성]")
print(char_system_prompt[:500] + "...")

# 2단계: 플롯 생성 (JSON 형식)
plot_system_prompt = f"""당신은 숏폼 영상 콘텐츠 시나리오 작가입니다.
사용자의 요청을 {num_cuts}개 장면으로 나누어 {'스토리' if mode == 'story' else '광고 콘텐츠'}를 만들어주세요.

각 장면마다 다음을 포함해야 합니다:
- scene_id: scene_1, scene_2, ... 형식
- char_id: char_1, char_2 (등장인물 ID)
- expression: 표정 (happy, sad, excited, neutral, surprised, angry 등)
- pose: 포즈 (standing, sitting, walking, pointing, jumping 등)
- text: 대사 또는 해설 내용
- text_type: dialogue (대사) 또는 narration (해설)
- emotion: neutral, happy, sad, excited, angry, surprised 중 하나
- subtitle_position: top 또는 bottom
- duration_ms: 장면 지속시간 (보통 4000-6000)

**중요**:
- 반드시 JSON 형식으로만 출력하세요
- expression과 pose는 이미지 생성에 사용됩니다
"""

print("\n[2단계: 플롯 생성 (JSON 형식)]")
print(plot_system_prompt[:500] + "...")

print("\n" + "=" * 50)
print(f"사용자 프롬프트: {test_prompt}")
print("=" * 50)

# 캐릭터 및 플롯 생성 실행
print("\n🔄 캐릭터 및 플롯 생성 중...")
characters_path, plot_json_path = generate_plot_with_characters(
    run_id=run_id,
    prompt=test_prompt,
    num_characters=num_characters,
    num_cuts=num_cuts,
    mode=mode,
)

print(f"\n✅ 캐릭터 JSON 생성 완료: {characters_path}")
print(f"✅ 플롯 JSON 생성 완료: {plot_json_path}")

GPT-4o-mini에게 전달되는 프롬프트 (2단계)
[1단계: 캐릭터 정의 생성]
당신은 숏폼 영상 콘텐츠의 캐릭터 디자이너입니다.
사용자의 요청에 맞는 2명의 캐릭터를 만들어주세요.

각 캐릭터마다 다음 정보를 JSON 형식으로 제공하세요:
- char_id: char_1, char_2, ... 형식
- name: 캐릭터 이름 (창의적으로)
- appearance: 외형 묘사 (이미지 생성에 사용됩니다. 상세하게 작성)
- personality: 성격/특징
- voice_profile: "default" (고정값)
- seed: char_1은 1002, char_2는 1003, ... 순서대로

**중요**:
- 반드시 JSON 형식으로만 출력하세요
- appearance는 이미지 생성 프롬프트로 사용되므로 시각적 특징을 상세히 작성
...

[2단계: 플롯 생성 (JSON 형식)]
당신은 숏폼 영상 콘텐츠 시나리오 작가입니다.
사용자의 요청을 5개 장면으로 나누어 스토리를 만들어주세요.

각 장면마다 다음을 포함해야 합니다:
- scene_id: scene_1, scene_2, ... 형식
- char_id: char_1, char_2 (등장인물 ID)
- expression: 표정 (happy, sad, excited, neutral, surprised, angry 등)
- pose: 포즈 (standing, sitting, walking, pointing, jumping 등)
- text: 대사 또는 해설 내용
- text_type: dialogue (대사) 또는 narration (해설)
- emotion: neutral, happy, sad, excited, angry, surprised 중 하나
- subtitle_position: top 또는 bottom
- duration_ms: 장면 지속시간 (보통 4000-6000)

**중요**:
- 반드시 ...

사용자 프롬프트: 우주를 여행하는 용감한 고양이의 모험

🔄 캐릭터 및 플롯 생성 중..

In [7]:
# 생성된 파일 확인 (스키마 v3)
import pandas as pd
import json

print("=" * 50)
print("1. 생성된 characters.json 내용")
print("=" * 50)

with open(characters_path, "r", encoding="utf-8") as f:
    characters_data = json.load(f)

for char in characters_data["characters"]:
    print(f"\n[{char['char_id']}] {char['name']}")
    print(f"  외형: {char['appearance']}")
    print(f"  성격: {char['personality']}")
    print(f"  음성: {char['voice_profile']}")
    print(f"  Seed: {char['seed']}")

print("\n" + "=" * 50)
print("2. 생성된 plot.json 내용 (v3 스키마)")
print("=" * 50)

# Read plot.json
with open(plot_json_path, "r", encoding="utf-8") as f:
    plot_data = json.load(f)

# Convert scenes to dataframe for display
df = pd.DataFrame(plot_data["scenes"])
print(df.to_string())

print(f"\n총 {len(plot_data['scenes'])}개 장면 생성됨")

print("\n" + "=" * 50)
print("스키마 v3 특징")
print("=" * 50)
print("✅ characters.json: 캐릭터 외형 및 설정 (appearance, personality, seed)")
print("✅ plot.json: 장면별 표정/포즈 제어 (expression, pose)")
print("✅ char_name 제거: characters.json에서 char_id로 참조")
print("✅ 이미지 생성: appearance + expression + pose 조합")

1. 생성된 characters.json 내용

[char_1] 스타캣
  외형: 짙은 남색 털을 가진 고양이로, 눈은 반짝이는 은색으로 빛나며, 우주복을 입고 있다. 우주복은 별무늬가 새겨져 있고, 작은 헬멧이 머리를 감싸고 있다. 꼬리는 길고 유연하여, 우주복의 뒷면에 장착된 제트팩과 연결되어 있다.
  성격: 호기심 많고 용감한 성격으로, 새로운 별과 행성을 탐험하는 것을 좋아한다. 또한, 친구들을 위해 언제나 친절하고 배려 깊은 태도를 유지한다.
  음성: default
  Seed: 1002

[char_2] 코스모
  외형: 작고 통통한 고양이로, 털 색깔은 밝은 노란색이며, 큰 초록색 눈을 가지고 있다. 작은 날개가 달린 우주 헬멧을 쓰고 있고, 다리는 짧지만 빠르게 움직일 수 있는 힘이 있다. 우주복은 귀여운 외계 생물 그림으로 장식되어 있다.
  성격: 장난꾸러기이지만, 위험한 상황에서도 침착함을 잃지 않는 성격이다. 친구와 함께 모험을 즐기며, 항상 긍정적인 에너지를 발산한다.
  음성: default
  Seed: 1003

2. 생성된 plot.json 내용 (v3 스키마)
  scene_id char_id expression         pose              text text_type    emotion subtitle_position  duration_ms
0  scene_1  char_1    excited     standing  안녕! 나는 용감한 고양이야!  dialogue      happy               top         5000
1  scene_2  char_2      happy      walking         우주로 출발하자!  dialogue    excited            bottom         4500
2  scene_3  char_1  surprised   looking_up        와! 저 별을 봐!  dialogue  surprised               

In [8]:
# plot.json + characters.json → layout.json 변환 (스키마 v3)
from app.utils.json_converter import convert_plot_to_json

print("🔄 characters.json + plot.json → layout.json 변환 중...")

json_path = convert_plot_to_json(
    plot_json_path=str(plot_json_path),
    run_id=run_id,
    art_style="파스텔 수채화",
    music_genre="ambient",
)

print(f"✅ JSON 생성 완료: {json_path}")

print("\n💡 변환 과정:")
print("  1. characters.json 읽기 → Character 객체")
print("  2. plot.json 읽기 → Scene 객체")
print("  3. 이미지 프롬프트 생성: appearance + expression + pose")
print("  4. layout.json 저장")

🔄 characters.json + plot.json → layout.json 변환 중...
✅ JSON 생성 완료: app/data/outputs/20251105_1738_우주를여행하는용/layout.json

💡 변환 과정:
  1. characters.json 읽기 → Character 객체
  2. plot.json 읽기 → Scene 객체
  3. 이미지 프롬프트 생성: appearance + expression + pose
  4. layout.json 저장


In [9]:
# JSON 내용 확인 (스키마 v4: texts 통합)
with open(json_path, "r", encoding="utf-8") as f:
    layout = json.load(f)

print("=" * 50)
print("JSON 레이아웃 구조 (v4)")
print("=" * 50)

print(f"\nProject ID: {layout['project_id']}")
print(f"Title: {layout['title']}")
print(f"Total Duration: {layout['timeline']['total_duration_ms']}ms")

print(f"\n캐릭터 ({len(layout['characters'])}명):")
for char in layout["characters"]:
    print(f"  - {char['name']} (ID: {char['char_id']}, Seed: {char['seed']})")
    print(f"    Persona: {char['persona'][:50]}...")

print(f"\n씬 ({len(layout['scenes'])}개):")
for scene in layout["scenes"]:
    print(f"\n  [{scene['scene_id']}] ({scene['duration_ms']}ms)")
    print(f"    이미지 슬롯: {len(scene['images'])}개")

    # 이미지 프롬프트 확인 (appearance + expression + pose)
    if scene["images"]:
        img = scene["images"][0]
        print(f"      이미지 URL: {img['image_url']}")
        print(f"      ref_id: {img.get('ref_id', 'N/A')}")

    # 텍스트 통합 확인
    print(f"    텍스트: {len(scene.get('texts', []))}개")
    for text_line in scene.get("texts", []):
        print(f"      [{text_line.get('text_type', 'N/A')}] {text_line['text'][:50]}...")
        print(f"      위치: {text_line.get('position', 'N/A')}")

print("\n" + "=" * 50)
print("스키마 v4 주요 특징")
print("=" * 50)
print("✅ characters.json 분리: 캐릭터 외형 재사용")
print("✅ plot.json expression/pose: 씬별 표정/포즈 제어")
print("✅ texts 통합: dialogue + subtitles → texts 배열")
print("✅ text_type 필드: dialogue/narration 구분")
print("✅ text_type=dialogue → 큰따옴표 자동 추가")

JSON 레이아웃 구조 (v4)

Project ID: 20251105_1738_우주를여행하는용
Title: AutoShorts 20251105_1738_우주를여행하는용
Total Duration: 24500ms

캐릭터 (2명):
  - 스타캣 (ID: char_1, Seed: 1002)
    Persona: 호기심 많고 용감한 성격으로, 새로운 별과 행성을 탐험하는 것을 좋아한다. 또한, 친구들을...
  - 코스모 (ID: char_2, Seed: 1003)
    Persona: 장난꾸러기이지만, 위험한 상황에서도 침착함을 잃지 않는 성격이다. 친구와 함께 모험을 즐기...

씬 (5개):

  [scene_1] (5000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_1
    텍스트: 1개
      [dialogue] "안녕! 나는 용감한 고양이야!"...
      위치: top

  [scene_2] (4500ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_2
    텍스트: 1개
      [dialogue] "우주로 출발하자!"...
      위치: bottom

  [scene_3] (4000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_1
    텍스트: 1개
      [dialogue] "와! 저 별을 봐!"...
      위치: top

  [scene_4] (5000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_2
    텍스트: 1개
      [dialogue] "우주에서의 모험은 정말 멋져!"...
      위치: bottom

  [scene_5] (6000ms)
    이미지 슬롯: 1개
      이미지 URL: 
      ref_id: char_1
    텍스트: 1개
      [dialogue] "모든 것을 정복

## 2. 이미지 생성 테스트 (Gemini Nano Banana / ComfyUI)

이미지 생성 프로바이더:
- **Gemini** (권장): Google Gemini 2.5 Flash Image (일명 Nano Banana), API 키만으로 즉시 사용 가능
- **ComfyUI**: 로컬 실행, 무료 (GPU 필요, `http://localhost:8188`)

In [10]:
# 이미지 생성 클라이언트 초기화
from app.config import settings

print("=" * 50)
print("이미지 생성 프로바이더 확인")
print("=" * 50)

provider = settings.IMAGE_PROVIDER
print(f"\n설정된 프로바이더: {provider}")

if provider == "gemini":
    # Gemini (Nano Banana) 설정 확인
    has_api_key = bool(settings.GEMINI_API_KEY)

    print(f"GEMINI_API_KEY: {'✅ 설정됨' if has_api_key else '❌ 미설정'}")

    if has_api_key:
        from app.providers.images.gemini_image_client import GeminiImageClient

        image_client = GeminiImageClient(api_key=settings.GEMINI_API_KEY)
        print(f"\n✅ Gemini (Nano Banana) 클라이언트 초기화 완료")
    else:
        print(f"\n⚠️  Gemini API 키가 설정되지 않음. .env 파일에 추가하세요:")
        print(f"   GEMINI_API_KEY=your_api_key")
        print(f"   (Google AI Studio에서 발급: https://ai.google.dev)")
        image_client = None

elif provider == "comfyui":
    # ComfyUI 연결 확인
    import httpx

    try:
        response = httpx.get(f"{settings.COMFY_URL}/system_stats", timeout=2.0)
        if response.status_code == 200:
            from app.providers.images.comfyui_client import ComfyUIClient

            image_client = ComfyUIClient()
            print(f"\n✅ ComfyUI 클라이언트 초기화 완료: {settings.COMFY_URL}")
        else:
            raise Exception("ComfyUI not responding")
    except:
        print(f"\n⚠️  ComfyUI 서버 연결 실패: {settings.COMFY_URL}")
        print(f"   1. ComfyUI 실행 여부 확인")
        print(f"   2. 또는 Gemini로 변경: .env에서 IMAGE_PROVIDER=gemini")
        image_client = None
else:
    print(f"\n❌ 알 수 없는 프로바이더: {provider}")
    image_client = None

이미지 생성 프로바이더 확인

설정된 프로바이더: gemini
GEMINI_API_KEY: ✅ 설정됨

✅ Gemini (Nano Banana) 클라이언트 초기화 완료


In [11]:
# 첫 번째 씬의 이미지 생성 테스트 (스키마 v3)
scene = layout["scenes"][0]
char = layout["characters"][0]

print("=" * 50)
print("이미지 생성 테스트")
print("=" * 50)

# plot.json에서 expression/pose 읽기
with open(plot_json_path, "r", encoding="utf-8") as f:
    plot_data = json.load(f)

first_scene = plot_data["scenes"][0]

# 프롬프트 구성 (v3 방식)
art_style = layout["metadata"]["art_style"]
# characters.json에서 appearance 가져오기
with open(characters_path, "r", encoding="utf-8") as f:
    characters_data = json.load(f)
    char_data = next(c for c in characters_data["characters"] if c["char_id"] == char["char_id"])
    appearance = char_data["appearance"]

expression = first_scene.get("expression", "neutral")
pose = first_scene.get("pose", "standing")
char_seed = char["seed"]
bg_seed = scene["bg_seed"]

# 이미지 프롬프트: appearance + expression + pose
if expression != "none" and pose != "none":
    image_prompt = f"{art_style}, {appearance}, {expression} expression, {pose} pose"
else:
    image_prompt = f"{art_style}, {appearance}"

print(f"캐릭터: {char['name']}")
print(f"Appearance: {appearance}")
print(f"Expression: {expression}")
print(f"Pose: {pose}")
print(f"Character Seed: {char_seed}")
print(f"Background Seed: {bg_seed}")
print(f"\n최종 프롬프트: {image_prompt}")

if image_client:
    print(f"\n🔄 이미지 생성 중 ({provider} 사용)...")

    try:
        if provider == "gemini":
            image_path = image_client.generate_image(
                prompt=image_prompt,
                seed=char_seed,
                width=512,
                height=768,  # 9:16 ratio
                output_prefix=f"app/data/outputs/{run_id}/scene_1_center",
            )
        elif provider == "comfyui":
            image_path = image_client.generate_image(
                prompt=image_prompt,
                seed=char_seed,
                output_prefix=f"app/data/outputs/{run_id}/scene_1_center",
            )

        print(f"\n✅ 이미지 생성 완료: {image_path}")

        # 이미지 표시 (Jupyter에서)
        from IPython.display import Image, display

        if Path(image_path).exists():
            display(Image(filename=image_path, width=400))

    except Exception as e:
        print(f"\n❌ 이미지 생성 실패: {e}")
        import traceback

        traceback.print_exc()
else:
    print(f"\n⚠️  이미지 클라이언트가 초기화되지 않음")
    print(f"   .env 파일에서 IMAGE_PROVIDER와 API 키를 확인하세요")

print("\n💡 v3 스키마:")
print("  - appearance (characters.json): 기본 외형")
print("  - expression (plot.json): 씬별 표정")
print("  - pose (plot.json): 씬별 포즈")
print("  → 조합하여 세밀한 이미지 제어 가능")

이미지 생성 테스트
캐릭터: 스타캣
Appearance: 짙은 남색 털을 가진 고양이로, 눈은 반짝이는 은색으로 빛나며, 우주복을 입고 있다. 우주복은 별무늬가 새겨져 있고, 작은 헬멧이 머리를 감싸고 있다. 꼬리는 길고 유연하여, 우주복의 뒷면에 장착된 제트팩과 연결되어 있다.
Expression: excited
Pose: standing
Character Seed: 1002
Background Seed: 2001

최종 프롬프트: 파스텔 수채화, 짙은 남색 털을 가진 고양이로, 눈은 반짝이는 은색으로 빛나며, 우주복을 입고 있다. 우주복은 별무늬가 새겨져 있고, 작은 헬멧이 머리를 감싸고 있다. 꼬리는 길고 유연하여, 우주복의 뒷면에 장착된 제트팩과 연결되어 있다., excited expression, standing pose

🔄 이미지 생성 중 (gemini 사용)...


Gemini image generation error: [Errno 2] No such file or directory: 'backend/app/data/outputs/images/app/data/outputs/20251105_1738_우주를여행하는용/scene_1_center.png'



❌ 이미지 생성 실패: [Errno 2] No such file or directory: 'backend/app/data/outputs/images/app/data/outputs/20251105_1738_우주를여행하는용/scene_1_center.png'

💡 v3 스키마:
  - appearance (characters.json): 기본 외형
  - expression (plot.json): 씬별 표정
  - pose (plot.json): 씬별 포즈
  → 조합하여 세밀한 이미지 제어 가능


Traceback (most recent call last):
  File "/var/folders/xt/9007rybs7kdb25r3g6xbrq840000gn/T/ipykernel_76873/1742292043.py", line 47, in <module>
    image_path = image_client.generate_image(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yuli/Documents/01.AI_Workspace/Kurz_Studio_AI/backend/app/providers/images/gemini_image_client.py", line 113, in generate_image
    with open(output_path, "wb") as f:
         ^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'backend/app/data/outputs/images/app/data/outputs/20251105_1738_우주를여행하는용/scene_1_center.png'


## 3. 음악 생성 테스트 (ElevenLabs / Mubert)

ElevenLabs Sound Effects API를 우선 사용하고, 없으면 Mubert를 폴백으로 사용합니다.

In [16]:
# Music 클라이언트 선택 (ElevenLabs 우선, Mubert 폴백)
from app.config import settings

if settings.ELEVENLABS_API_KEY:
    from app.providers.music.elevenlabs_music_client import ElevenLabsMusicClient

    music_client = ElevenLabsMusicClient(api_key=settings.ELEVENLABS_API_KEY)
    print("✅ ElevenLabs Music 클라이언트 초기화 완료")
    print("💡 ElevenLabs Sound Effects API 사용 (TTS와 통합, 저렴)")
elif settings.MUBERT_LICENSE:
    from app.providers.music.mubert_client import MubertClient

    music_client = MubertClient(api_key=settings.MUBERT_LICENSE)
    print("✅ Mubert 클라이언트 초기화 완료 (폴백)")
else:
    from app.providers.music.stub_client import StubMusicClient

    music_client = StubMusicClient()
    print("⚠️  Stub 모드로 실행 (음악 API 없음)")

ModuleNotFoundError: No module named 'requests'

In [17]:
# 배경음악 생성
genre = layout["metadata"]["music_genre"]
duration_ms = layout["timeline"]["total_duration_ms"]
duration_sec = duration_ms / 1000

print("=" * 50)
print("음악 생성 파라미터")
print("=" * 50)
print(f"장르: {genre}")
print(f"길이: {duration_sec}초 ({duration_ms}ms)")
print(f"무드: cinematic")

# ElevenLabs 사용 시 프롬프트 표시
if hasattr(music_client, "_build_music_prompt"):
    # ElevenLabs 클라이언트인 경우
    prompt = music_client._build_music_prompt(genre, "cinematic", duration_sec)
    print(f"\n[ElevenLabs Sound Effects API 프롬프트]")
    print(f"{prompt}")
    print(f"\nAPI 파라미터:")
    print(f"  - duration_seconds: {min(duration_sec, 22)} (최대 22초)")
    print(f"  - prompt_influence: 0.3")
elif hasattr(music_client, "generate"):
    # Mubert 클라이언트인 경우
    print(f"\n[Mubert API 파라미터]")
    print(f"  - genre: {genre}")
    print(f"  - mood: cinematic, dreamy")

print("\n🔄 음악 생성 중...")

try:
    audio_path = music_client.generate_music(
        genre=genre,
        mood="cinematic",
        duration_ms=duration_ms,
        output_filename=f"app/data/outputs/{run_id}/global_bgm.mp3",
    )

    print(f"\n✅ 음악 생성 완료: {audio_path}")

    # 오디오 재생 (Jupyter에서)
    from IPython.display import Audio, display

    if Path(audio_path).exists():
        display(Audio(filename=audio_path))

except Exception as e:
    print(f"\n❌ 음악 생성 실패: {e}")
    import traceback

    traceback.print_exc()

음악 생성 파라미터
장르: ambient
길이: 27.0초 (27000ms)
무드: cinematic


NameError: name 'music_client' is not defined

## 4. 음성 생성 테스트 (ElevenLabs / PlayHT)

TTS API를 사용하여 캐릭터 대사를 생성합니다.

In [ ]:
# TTS 클라이언트 선택
from app.config import settings

if settings.ELEVENLABS_API_KEY:
    from app.providers.tts.elevenlabs_client import ElevenLabsClient

    tts_client = ElevenLabsClient(api_key=settings.ELEVENLABS_API_KEY)
    print("✅ ElevenLabs 클라이언트 초기화 완료")
elif settings.PLAYHT_API_KEY:
    from app.providers.tts.playht_client import PlayHTClient

    tts_client = PlayHTClient(
        user_id=settings.PLAYHT_USER_ID, api_key=settings.PLAYHT_API_KEY
    )
    print("✅ PlayHT 클라이언트 초기화 완료")
else:
    from app.providers.tts.stub_client import StubTTSClient

    tts_client = StubTTSClient()
    print("⚠️  Stub 모드로 실행 (TTS 없음)")

In [19]:
# 첫 번째 대사 생성
dialogue = scene["dialogue"][0]

print("=" * 50)
print("음성 생성 파라미터")
print("=" * 50)
print(f"캐릭터: {char['name']}")
print(f"대사: {dialogue['text']}")
print(f"감정: {dialogue['emotion']}")
print(f"Voice ID: {char.get('voice_profile', 'default')}")

print("\n🔄 음성 생성 중...")

try:
    voice_path = tts_client.generate_speech(
        text=dialogue["text"],
        voice_id=char.get("voice_profile", "default"),
        emotion=dialogue["emotion"],
        output_filename=f"app/data/outputs/{run_id}/scene_1_line_1.mp3",
    )

    print(f"\n✅ 음성 생성 완료: {voice_path}")

    # 오디오 재생
    from IPython.display import Audio, display

    if Path(voice_path).exists():
        display(Audio(filename=voice_path))

except Exception as e:
    print(f"\n❌ 음성 생성 실패: {e}")

음성 생성 파라미터
캐릭터: 코스모냥이
대사: 나는 우주를 탐험할 준비가 되었어! 텔레포트 기계를 작동시켜 보자! 
감정: excited
Voice ID: default

🔄 음성 생성 중...

❌ 음성 생성 실패: name 'tts_client' is not defined


## 5. 통합 테스트 - 전체 씬 생성

모든 씬의 에셋을 생성하고 JSON을 업데이트합니다.

In [ ]:
# 첫 번째 씬의 이미지 생성 테스트
scene = layout["scenes"][0]
char = layout["characters"][0]

print("=" * 50)
print("이미지 생성 프롬프트")
print("=" * 50)

# 프롬프트 구성
art_style = layout["metadata"]["art_style"]
persona = char["persona"]
char_seed = char["seed"]
bg_seed = scene["bg_seed"]

image_prompt = f"{art_style}, {persona}, standing, neutral expression"

print(f"캐릭터: {char['name']}")
print(f"Persona: {persona}")
print(f"Character Seed: {char_seed}")
print(f"Background Seed: {bg_seed}")
print(f"\n최종 프롬프트: {image_prompt}")

print("\n🔄 이미지 생성 중...")

try:
    image_path = image_client.generate_image(
        prompt=image_prompt,
        seed=char_seed,
        output_prefix=f"app/data/outputs/{run_id}/scene_1_center",
    )

    print(f"\n✅ 이미지 생성 완료: {image_path}")

    # 이미지 표시 (Jupyter에서)
    from IPython.display import Image, display

    if Path(image_path).exists():
        display(Image(filename=image_path, width=300))

except Exception as e:
    print(f"\n❌ 이미지 생성 실패: {e}")
    print("\n💡 ComfyUI가 실행 중인지 확인하세요: http://localhost:8188")
    import traceback

    traceback.print_exc()

In [ ]:
# 업데이트된 JSON 저장
updated_json_path = Path(json_path).parent / "layout_with_assets.json"
with open(updated_json_path, "w", encoding="utf-8") as f:
    json.dump(layout, f, indent=2, ensure_ascii=False)

print(f"✅ 업데이트된 JSON 저장: {updated_json_path}")

# 최종 결과 요약
print("\n" + "=" * 50)
print("최종 결과 요약")
print("=" * 50)

total_images = sum(len(s["images"]) for s in layout["scenes"])
total_voices = sum(len(s["dialogue"]) for s in layout["scenes"])

print(f"생성된 이미지: {total_images}개")
print(f"생성된 음성: {total_voices}개")
print(f"배경음악: 1개")
print(f"\n모든 에셋 저장 위치: app/data/outputs/{run_id}/")

## 6. 프롬프트 분석 및 최적화

생성된 프롬프트를 분석하고 개선 방안을 제안합니다.

In [ ]:
# 프롬프트 분석 (스키마 v4: texts 통합)
print("=" * 50)
print("프롬프트 분석 (v4)")
print("=" * 50)

print("\n1. GPT 프롬프트 (2단계 생성)")
print("-" * 50)
print("1단계: 캐릭터 정의 생성 (characters.json)")
print("  - char_id, name, appearance, personality, voice_profile, seed")
print("  - appearance: 이미지 생성 기본 프롬프트")
print("\n2단계: 플롯 생성 (plot.json)")
print(
    "  - scene_id, char_id, expression, pose, text, text_type, emotion, subtitle_position, duration_ms"
)
print("  - expression/pose: 씬별 이미지 제어")

print("\n2. 이미지 생성 프롬프트 (v4: appearance + expression + pose)")
print("-" * 50)

# plot.json 읽기
with open(plot_json_path, "r", encoding="utf-8") as f:
    plot_data = json.load(f)

# characters.json 읽기
with open(characters_path, "r", encoding="utf-8") as f:
    characters_data = json.load(f)

for char in layout["characters"]:
    # plot.json에서 해당 캐릭터의 첫 번째 씬 정보 가져오기
    char_scenes = [s for s in plot_data["scenes"] if s["char_id"] == char["char_id"]]

    if char_scenes:
        first_scene = char_scenes[0]
        expression = first_scene.get("expression", "neutral")
        pose = first_scene.get("pose", "standing")

        # characters.json에서 appearance 가져오기
        char_data = next(c for c in characters_data["characters"] if c["char_id"] == char["char_id"])
        appearance = char_data["appearance"]

        if expression != "none" and pose != "none":
            prompt = f"{art_style}, {appearance}, {expression} expression, {pose} pose"
        else:
            prompt = f"{art_style}, {appearance}"

        print(f"[{char['name']}] {prompt[:80]}...")
        print(f"  구성: appearance + expression + pose")
        print(f"  Seed: {char['seed']}")

print("\n3. 음성 생성 파라미터")
print("-" * 50)
for scene in layout["scenes"]:
    for text_line in scene.get("texts", []):
        char = next(
            c for c in layout["characters"] if c["char_id"] == text_line["char_id"]
        )
        print(f"[{char['name']}] {text_line['text'][:30]}...")
        print(f"  타입: {text_line.get('text_type', 'N/A')}")
        print(f"  감정: {text_line['emotion']}")
        print(f"  길이: {len(text_line['text'])} 문자")

print("\n4. 음악 생성 프롬프트")
print("-" * 50)
genre = layout["metadata"]["music_genre"]
duration_ms = layout["timeline"]["total_duration_ms"]
duration_sec = duration_ms / 1000

print(f"[ElevenLabs Sound Effects API 권장]")
print(
    f"프롬프트 예시: calm and peaceful {genre} atmospheric background music, instrumental, no vocals, looping"
)
print(f"길이: {duration_sec}초 (최대 22초)")
print(f"API 파라미터: duration_seconds, prompt_influence=0.3")

print("\n" + "=" * 50)
print("스키마 v4 최적화 포인트")
print("=" * 50)
print("1. 캐릭터 외형 재사용: characters.json → 모든 씬에서 일관성")
print("2. 씬별 세밀 제어: expression + pose → 같은 캐릭터, 다양한 연출")
print("3. 이미지 프롬프트 자동 조합: appearance + expression + pose")
print("4. 음악: ElevenLabs 권장 (TTS와 통합, 저렴)")
print("5. JSON 스키마: text_type으로 dialogue/narration 명확히 구분")
print("6. texts 통합: dialogue + subtitles → 하나의 texts 배열로 관리")

## 7. 정리 및 다음 단계

생성된 모든 파일과 다음 단계를 확인합니다.

In [ ]:
# 생성된 파일 목록
output_dir = Path(f"app/data/outputs/{run_id}")

print("=" * 50)
print(f"생성된 파일 목록 ({output_dir})")
print("=" * 50)

if output_dir.exists():
    files = sorted(output_dir.glob("*"))

    for file in files:
        size_mb = file.stat().st_size / (1024 * 1024)
        print(f"{file.name:40s} {size_mb:>8.2f} MB")

    print(f"\n총 {len(files)}개 파일")
else:
    print("⚠️  출력 디렉토리가 없습니다.")

print("\n" + "=" * 50)
print("다음 단계")
print("=" * 50)
print("1. Director Agent로 영상 합성 (MoviePy)")
print("2. QA Agent로 품질 검수")
print("3. 최종 영상 확인")
print("\n💡 전체 파이프라인 실행은 FastAPI + Celery로 진행하세요.")